# Grado en ciencia de datos - Big Data

# Práctica 2 - Parte II - Ejercicios SparkSQL

En este segundo notebook de la práctica 2 se deberán realizar varios ejercicios haciendo uso de los DataFrames de Spark. 

Ten en cuenta que una vez tengas en marcha Spark, podrás visualizar la evolución de cada trabajo de Spark en  <http://localhost:4040>

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Ejemplo pySparkSQL") \
    .config("spark.sql.warehouse.dir", "file:///D:/tmp/spark-warehouse") \
    .getOrCreate()

sc = spark.sparkContext

25/12/07 13:25:54 WARN Utils: Your hostname, MacBook-Pro-de-Sergio.local resolves to a loopback address: 127.0.0.1; using 192.168.1.13 instead (on interface en0)
25/12/07 13:25:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/07 13:25:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 49796)
Traceback (most recent call last):
  File "/opt/miniconda3/envs/py311ml/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/miniconda3/envs/py311ml/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/miniconda3/envs/py311ml/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/miniconda3/envs/py311ml/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/opt/miniconda3/envs/py311ml/lib/python3.11/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/opt/miniconda3/envs/py311ml/lib/python3.11/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func

Otros imports necesarios:

In [2]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from test_helper import Test
from pyspark.sql import functions as F
from pyspark.sql import Row

## Ejercicio 1. Contar palabras

Crear una función MapReduce en Spark llamada `cuentaPalabras(filePath)` que cuente cuántas veces aparece cada palabra en un documento o conjunto de documentos de entrada. Utiliza DataFrames.

**Entrada:** Documento o documentos

**Salida:** (Palabra, Número de apariciones)

Pasos a seguir:
1. Leer el fichero. Cada línea es un elemento del DataFrame (columna value).
2. Dividir las líneas en palabras.
3. Filtrar palabras vacías.
4. Contar las occurrencias de cada palabra.
5. Devolver al driver las 10 palabras más repetidas


In [3]:
def cuentaPalabras(filePath):
    dfText = spark.read.text(filePath)
    
    return dfText.select(F.explode(F.split(dfText["value"], " ")).alias("word")) \
                 .filter("word != ''") \
                 .groupBy("word").count() \
                 .sort(F.desc("count")) \
                 .take(10) 

El programa debe ser capaz de pasar los siguientes tests

In [4]:
top10Quijote = cuentaPalabras("./datos/pg2000.txt")
print(top10Quijote)
Test.assertEquals(top10Quijote, [(u'que', 19429), (u'de', 17988), (u'y', 15894), (u'la', 10200), 
                          (u'a', 9575), (u'el', 7957), (u'en', 7898), (u'no', 5611), 
                          (u'se', 4690), (u'los', 4680)],
                  'Resultado incorrecto')

[Row(word='que', count=19429), Row(word='de', count=17988), Row(word='y', count=15894), Row(word='la', count=10200), Row(word='a', count=9575), Row(word='el', count=7957), Row(word='en', count=7898), Row(word='no', count=5611), Row(word='se', count=4690), Row(word='los', count=4680)]
1 test passed.


### Parte 2 - Mejorando la cuenta de palabras
Utiliza la función `eliminarPuntuacion(text)` para contar todas las palabras igual independientemente de las mayúsculas, los signos de puntuación etc. Utilizar DataFrames.

Pasos a seguir:
1. Leer el fichero. Cada línea es un elemento del DataFrame (columna value).
2. Eliminar los signos de puntuación.
2. Dividir las líneas en palabras.
3. Filtrar palabras vacías.
4. Contar las occurrencias de cada palabra.
5. Devolver al driver las 10 palabras más repetidas

In [5]:
from pyspark.sql.functions import regexp_replace, trim, col, lower
def eliminaSignosPuntuacion(column):
    """Elimina los signos de puntuación, pasa las palabras a minúsculas y elimina los espacios de más antes y después
    Args:
        column (Column): Una columna con una frase

    Returns:
        Column: Una columna llamada frase con la frase original limpia
    """
    return lower(trim(regexp_replace(column, r'[^0-9a-zA-ZñÑáéíóúÁÉÍÓÚ ]+', ''))).alias('frase')

fraseDF = spark.createDataFrame([(u'Hola!, Qué tal?',),
                                         (u' Sin barras_bajas!',),
                                         (u' *      Elimina puntación y espacios  * ,',)], ['frase'])
fraseDF.show(truncate=False)
(fraseDF
       .select(eliminaSignosPuntuacion(col('frase')))
       .show(truncate=False))

+-----------------------------------------+
|frase                                    |
+-----------------------------------------+
|Hola!, Qué tal?                          |
| Sin barras_bajas!                       |
| *      Elimina puntación y espacios  * ,|
+-----------------------------------------+

+----------------------------+
|frase                       |
+----------------------------+
|hola qué tal                |
|sin barrasbajas             |
|elimina puntación y espacios|
+----------------------------+



In [6]:
def cuentaPalabras(filePath):
    dfText = spark.read.text(filePath)
    
    return dfText.select(eliminaSignosPuntuacion(col("value"))) \
                 .select(F.explode(F.split(col("frase"), " ")).alias("word")) \
                 .filter("word != ''") \
                 .groupBy("word").count() \
                 .sort(F.desc("count")) \
                 .take(10) 

In [7]:
top10Quijote = cuentaPalabras("./datos/pg2000.txt")
Test.assertEquals(top10Quijote, [(u'que', 20626), (u'de', 18217), (u'y', 18188), (u'la', 10363), (u'a', 9880),
                                 (u'en', 8241), (u'el', 8210), (u'no', 6345), (u'los', 4748), (u'se', 4690)],
                  'Resultado incorrecto')

1 test passed.


## Ejercicio 2. Histograma de repeticiones

Obtener un histograma del número de repeticiones de las palabras, es decir, cuántas palabras se repiten X veces:
* 1 vez – 3 palabras
* 2 veces – 10 palabras
* 3 veces – 20 palabras
* ...

Para ello crea una función MapReduce en Spark llamada `histogramaRepeticiones(filePath)`. Esta función NO debe hacerr uso de la función `cuentaPalabras(filePath)` del programa anterior aunque compartirá parte de su código. Todo debe realizarse mediante DafaFrames salvo el `collect()` final que devolverá una lista. La lista debe de estar ordenada por el número de veces. Continúa utilizando la función `eliminarPuntuacion(text)` para contar igual todas las palabras.

**Entrada:** Documento o documentos

**Salida:** (X veces, Número de palabras)


In [8]:
def histogramaRepeticiones(filePath):
    dfText = spark.read.text(filePath)
    
    return dfText.select(eliminaSignosPuntuacion(col("value"))) \
                 .select(F.explode(F.split(col("frase"), " ")).alias("word")) \
                 .filter("word != ''") \
                 .groupBy("word").count()\
                 .groupBy("count").count()\
                 .sort(F.asc("count")) \
                 .take(20) 

El programa debe ser capaz de pasar los siguientes tests

In [9]:
histQuijote = histogramaRepeticiones("./datos/pg2000.txt")
Test.assertEquals(histQuijote[:10], [(1, 11583), (2, 3664), (3, 1860), (4, 1147), (5, 780), (6, 552), 
                                     (7, 422), (8, 334), (9, 261), (10, 227)],
                  'Resultado incorrecto')

1 test passed.


Podemos realizar un gráficos a partir de los datos con matplotlib

In [10]:
### Rellenar

## Ejercicio 3. Análisis sobre los tweets de las elecciones de EEUU de 2012

En lo que resta de práctica vamos a hacer uso del dataset disponible en la siguiente URL <https://datahub.io/dataset/twitter-2012-presidential-election>.

Este dataset contiene millones de tweets recogidos durante la campaña electoral de 2012 cuando se enfrentaban Romney (republicano) frente a Obama (demócrata). Se ha llegado a decir que una de las cosas que hizo ganar a Obama fue el uso del Big Data y el análisis de las redes sociales. De manera similar, nosotros vamos a trabajar con estos tweets para ver de lo que es capaz SparkSQL, desde la carga de datos a su análisis.

En nuestro caso, debido al tamaño del dataset utilizaremos solo la primera parte (400MB comprimidas y 3GB descomprimida). 

En primer lugar, leemos el dataset (en formato JSON) y lo cacheamos, para no tener que leerlo continuamente de disco.

In [11]:
datosJSON = spark.read.json('datos/tweets2012/cache-0.json').cache()

25/12/07 13:26:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


#### ¿Cuántos tweets tenemos disponibles para analizar?

In [12]:
nTweets = datosJSON.count()

25/12/07 13:26:03 WARN BlockManager: Block rdd_43_7 could not be removed as it was not found on disk or in memory
25/12/07 13:26:03 ERROR Executor: Exception in task 7.0 in stage 19.0 (TID 62)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:64)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:363)
	at org.apache.spark.sql.execution.columnar.ColumnBuilder$.ensureFreeSpace(ColumnBuilder.scala:167)
	at org.apache.spark.sql.execution.columnar.BasicColumnBuilder.appendFrom(ColumnBuilder.scala:73)
	at org.apache.spark.sql.execution.columnar.ComplexColumnBuilder.org$apache$spark$sql$execution$columnar$NullableColumnBuilder$$super$appendFrom(ColumnBuilder.scala:93)
	at org.apache.spark.sql.execution.columnar.NullableColumnBuilder.appendFrom(NullableColumnBuilder.scala:61)
	at org.apache.spark.sql.execution.columnar.NullableColumnBuilder.appendFrom$(NullableColumnBuilder.scala:54)
	at org.apache.spark.sql.execution.columnar.

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
Test.assertEquals(nTweets, 1000000, "El cálculo del número de tweets es incorrecto")

### Imprime el esquema del DataFrame y la primera fila para ver qué tipo de datos tenemos

In [ ]:
datosJSON.printSchema()

In [ ]:
datosJSON.first()

### Filtrado de campos
Como puedes observar, tenemos una gran cantidad de campos correspondientes a cada tweet. Sin embargo, no todos son interesantes o no nos van a ser últiles para trabajar en esta práctica. Por ello, será más sencillo si solo nos quedamos con los campos que van a ser útiles.

### Crea un nuevo DataFrame, denominado `df`, que contenga solo los datos correspondientes a las siguientes columnas
* id
* user.name
* user.followers_count
* text
* retweet_count
* place.country
* entities.user_mentions
* entities.hashtags
* created_at

In [ ]:
df = datosJSON.select('id', 'user.name', 'user.followers_count', 'text', 'retweet_count', 'place.country', 'entities.user_mentions', 'entities.hashtags', 'created_at')

In [ ]:
Test.assertEquals(df.columns, ['id', 'name', 'followers_count', 'text', 'retweet_count',
                               'country', 'user_mentions', 'hashtags', 'created_at'],
                 'Columnas seleccionadas incorrectas')

## Ejercicios básicos

### 1. Obtener la media de la longitud de los tweets por usuario ordenado de mayor a menor. ¿Quién es el usuario con mayor media de longitud? - considera solo aquellos que hayan publicado más de 100 tweets

Pasos a seguir:
1. Utiliza un select para obtener junto con cada nombre de usuario la longitud del tweet (usar función `length`)
2. Agrupar por nombre y obtener la media de la longitud y el conteo
3. Filtrar por conteo
5. Ordenar por media de la longitud
6. Obtener el nombre del primer usuario

In [ ]:
## Devolver solo el nombre para pasar el test
usuario = df.select(df.name, F.length(col("text")).alias("lenTweet")) \
            .groupBy("name").agg(F.count("lenTweet"), F.mean("lenTweet")) \
            .filter("count(lenTweet) > 100") \
            .sort(F.desc("avg(lenTweet)")) \
            .first()[0]#Devuelvo solo el nombre para pasar el test.

In [ ]:
Test.assertEquals(usuario, u'Cypress Gang', 'Usuario incorrecto')

### 2. Obtener el número de tweets por país para los tweets que tienen el país establecido


In [ ]:
# Si en alguna ejecución no pasa el test porque cambia las posiciones de Spain e Indonesia está bien
tweetsPorPaisDF = df.filter(df.country.isNotNull()) \
                    .groupBy("country").count() \
                    .sort(F.desc("count"))

In [ ]:
print (tweetsPorPaisDF.take(10))

In [ ]:
Test.assertEquals(tweetsPorPaisDF.take(10), [(u'United States', 5240), (u'Brasil', 1212), (u'United Kingdom', 386), (u'Germany', 223), 
                                             (u'Indonesia', 188), (u'Spain', 188), (u'Mexico', 174), (u'Italy', 150), (u'Canada', 121), 
                                             (u'France', 94)], 'Tweets por país incorrectos')

### 3. Obtener el listado de los usuarios más mencionados
A tener en cuenta:
* Las menciones a otros usuarios aparecen en el campo `user_mentions` que es un array. La forma de desempaquetar el array es mediante la función explode. Una vez aplicada, se puede realizar otro select para quedarnos solo con el nombre del usuario `name`.

In [ ]:
mencionUsuariosDF = df.select(F.explode("user_mentions").alias("fila")) \
                      .select(F.col('fila.name')) \
                      .groupBy("name").count() \
                      .sort(F.desc("count")) 

In [ ]:
Test.assertEquals(mencionUsuariosDF.take(10), [(u'Barack Obama', 13895), (u'Nicki Minaj', 9744), (u'YouTube', 5127), 
                                               (u'Mitt Romney', 4069), (u'Bill Maher', 3908), (u'ShareThis', 3876), 
                                               (u'2Chainz (Tity Boi)', 3862), (u'Most Funniest Man', 3059), 
                                               (u'PublicPolicyPolling', 2776), (u'Top Tweets \u2655', 2097)],
                  'Tweets por país incorrectos')

## Hashtags más populares

Vamos a realizar un análisis de los hashtags más populares

#### Imprime el esquema del DataFrame actual (`df`)

In [ ]:
df.printSchema()

#### Vamos a estudiar cuáles han sido los hashtags más populares
Para ello, vamos a seguir los siguientes pasos:
1. Los hashtags vienen dados por un array. La función `explode` nos permite desempaquetar el array y convertir cada el elemento del array en una fila. Denominaremos a esta columna 'tags'
2. Puedes observar (printSchema) como cada elemento de la columna tags está formado por el índice y el texto. Solo nos interesa el texto y además vamos a convertirlo a minúsculas (función `lower`). La columna resultante se llamará 'tag'.
3. Ya tenemos un DataFrame con todas las tags que han aparecido en los tweets que tenemos disponibles. Solo nos queda contar cuántas veces aparece cada tag (piensa como combinar `groupBy` con `count`).
4. Por último, nos gustaría obtener el DataFrame ordenado de mayor a menor número de veces que ha sido usado un tag. 
5. Cachea el DataFrame obtenido ya que lo usaremos un par de veces y nos evitaremos volver a calcularlo

In [ ]:
top10tags = df.select(F.explode("hashtags").alias("tags")) \
                      .select(F.lower(F.col('tags.text')).alias("text")) \
                      .groupBy("text").count() \
                      .sort(F.desc("count"))

top10tags.cache()

In [ ]:
Test.assertEquals(top10tags.take(10), [(u'obama', 30643), (u'usa', 30405), 
                              (u'tcot', 19116), (u'p2', 8608), 
                              (u'romney', 6171), (u'news', 4785), 
                              (u'gop', 4600), (u'obama2012', 4179), 
                              (u'teaparty', 4057), (u'somalia', 3636)],
                 'Tags obtenidas incorrectas')

Test.assertEquals(top10tags.is_cached, True, 'DataFrame no cacheado')

Los resultados obtenidos son curiosos, Obama es el hashtag más utilizado, seguido de cerca por USA, sin embargo, Romney, el candidato republicano aparece en muchos menos hashtags. ¿Tienen los republicanos menos presencia en twitter?

Aunque la respuesta aparentemente podría ser sí, es interesante ver que aparecen otros hashtags como 'tcot', ¿a qué se refiere? El próximo artículo nos da la respuesta: <http://www.ibtimes.com/what-does-tcot-mean-about-tcot-hashtag-top-conservatives-use-twitter-1109812>

En definitiva, los republicanos no se centraban tanto en el candidato, pero igualmente tenían presencia en twitter.

#### Hashtags y menciones a usuarios
Vamos a ver qué hashtags aparecen más veces junto con la mención a un mismo usuario. Esta vez nos olvidamos de pasarlo a mínusculas (no usar `lower`).

Queremos obtener un DataFrame en el que tengamos, el nombre del usuario mencionado, el texto de la hashtag y el conteo de cuántas veces ha aparecido conjuntamente. El DataFrame debe estar ordenado por conteo.

Recuerda que tanto los hashtags, como las menciones a usuarios están almacenadas en arrays que deben desempaquetarse. Para ello ten en cuenta que no puedes usar dos veces la función `explode` en el mismo select, y que por tanto debe de hacerse por separado.

In [ ]:
userHashtagDF = df.select(F.col("user_mentions").alias("mentions"), F.explode("hashtags").alias("tags")) \
                  .select(F.explode("mentions.name").alias("user"), F.col('tags.text').alias("text")) \
                  .groupBy("user", "text").count()\
                  .sort(F.desc("count"))

In [ ]:
### Si no pasa este test es posible que sea porque las siguientes líneas aparecen cambiadas:
#|     RCTV_CONTIGO|    ChávezCADUCÓ| 1233|
#|     RCTV_CONTIGO|   ChavezGranCob| 1175|
# Si solo es eso, está OK
Test.assertEquals(userHashtagDF.take(10), [(u'#TeamFollowBack', u'TeamFollowBack',  1487), (u'#TeamFollowBack', u'BillionDollarArt',  1458), 
                                           (u'#TeamFollowBack', u'USA',  1436), (u'#TeamFollowBack', u'NYC',  1372), 
                                           (u'RCTV_CONTIGO', u'Ch\xe1vezCADUC\xd3',  1231), (u'RCTV_CONTIGO', u'ChavezGranCob',  1173), 
                                           (u'CNN Breaking News', u'Obama',  834), (u'CNN Breaking News', u'Romney',  829), 
                                           (u'CNN Breaking News', u'CNNelections',  819), (u'Son of a Fratter', u'USA',  697)], 
                  'DataFrame de usuarios mencionados y hashtags incorrecto')

#### Presencia en los tweets de cada uno de los candidatos
En el apartado anterior sobre los hashtags solo nos hemos fijado en los hashtags, pero no hemos prestado atención al texto del mensaje. En el siguiente ejercicio vamos a tratar de ver en cuántos tweets estaba presente cada partido/candidato. Aunque debemos tener en cuenta que la mayor presencia no tiene porqué ser siempre buena (esto requeriría de un análisis mucho más profundo).

Para simplificar la tarea asumiremos que las palabras relacionadas con Obama/demócratas son: obama y democrat; y las relacionadas Romney/republicanos son: romney, republican y tcot.

Nuestro primer objetivo es crear un DataFrame en el que dispongamos de una columna que indique si menciona a Obama/demócratas  y otra si menciona a Romney/republicanos.

Para llevar a cabo esta tarea, vamos a seguir los siguientes pasos.
1. Crea un nuevo DataFrame a partir de `df`. A este DataFrame le vamos a añadir una nueva columna, llamada 'democrat', que nos indicará si en dicho tweet se ha mencionado alguna de las palabras correspondientes a los demócratas (obama, democrat). Si se ha mencionado alguna de ellas la columna tendrá el valor 'Democrat' y en otro caso el valor '-'. Para ello, haz uso de `withColumn` y las funciones `when/otherwise`. Considera siempre el texto en mínusculas (`lower`) para encontrar las coincidencias con `like`.
2. Al DataFrame que hemos generado en el pimer punto, vamos a añadir otra columna igual pero para el caso de las menciones de los republicanos, la nueva columna se llamará 'Republican'.
3. Obtener un DataFrame que nos indique qué porcentaje de los tweets mencionan algo que tienen que ver con los demócratas y en qué porcentaje no se les menciona. Columnas: democrat, porcentaje. El DataFrame se llamará democratDF. Para el cálculo del porcentaje, debemos utilizar el conteo total de tweets (nTweets).
4. Obtener un DataFrame que nos indique qué porcentaje de los tweets mencionan algo que tienen que ver con los republicanos y en qué porcentaje no se les menciona. Columnas: republican, porcentaje. El DataFrame se llamará republicanDF.
5. Por último, vamos a obtener un resumen más completo en el DataFrame partyDF. Para ello, vamos a añadir una nueva columna al DataFrame con democrat y republican que incluya la combinación de las dos columnas (`concat`). Posteriormente, obtendremos el porcentaje de tweets que corresponden a cada combinación. Así podemos analizar qué porcentaje de tweets mencionan solo a demócratas, solo a republicanos, a los dos a la vez o a ninguno.

**Nota: Todos los DataFrames deben estar ordenados de mayor a menor porcentaje**

In [ ]:
columnaDemocratDF = df.withColumn('democrat', F.when(F.lower(df.text).like('%obama%'), "Democrat") \
                      .otherwise(F.when(F.lower(df.text).like('%democrat%'), "Democrat") \
                      .otherwise("-")))

columnaRepublicanDemocratDF = columnaDemocratDF.withColumn('republican', F.when(F.lower(df.text).like('%romney%'), "Republican") \
                                               .otherwise(F.when(F.lower(df.text).like('%republican%'), "Republican") \
                                               .otherwise(F.when(F.lower(df.text).like('%tcot%'), "Republican") \
                                               .otherwise("-"))))

columnaRepublicanDemocratCombinadaDF = columnaRepublicanDemocratDF.select('republican', 'democrat')

democratDF = columnaDemocratDF.groupBy('democrat') \
                              .count() \
                              .select('democrat', (100*F.col('count')/nTweets).alias('count')) \
                              .sort('count', ascending=False)

republicanDF = columnaRepublicanDemocratDF.groupBy('republican') \
                                          .count() \
                                          .select('republican', (100*F.col('count')/nTweets).alias('count')) \
                                          .sort('count', ascending=False)

dfParty = columnaRepublicanDemocratCombinadaDF.withColumn('concatenated', F.concat(*['republican', 'democrat'])) \
                                              .groupBy('concatenated') \
                                              .count() \
                                              .select('concatenated', (100*F.col('count')/nTweets).alias('count')) \
                                              .sort('count', ascending=False)

# Mostrar dataframes
democratDF.show()
republicanDF.show()
dfParty.show()

In [ ]:
round_fun = lambda res: list(map(lambda k_v: (k_v[0], np.round(k_v[1], 4)), res))
Test.assertEquals(round_fun(democratDF.collect()), [(u'Democrat', 53.2483), (u'-', 46.7517)], 
                  'DataFrame de democratDF incorrecto')
Test.assertEquals(round_fun(republicanDF.collect()), [(u'-', 83.3746), (u'Republican', 16.6254)], 
                  'DataFrame de republicanDF incorrecto')
Test.assertEquals(round_fun(dfParty.collect()), [(u'--', 45.0071), (u'-Democrat', 38.3675), 
                                      (u'RepublicanDemocrat', 14.8808), (u'Republican-', 1.7446)], 
                  'DataFrame de dfParty incorrecto')

De nuevo el resultado deja ver que los demócratas tienen más presencia. Quizás sea porque los republicanos únicamente se dedicaban a criticarlos o porque realmente los demócratas se movieron mucho más en las redes sociales

# Explorar más el dataset de tweets y nuevas conclusiones

* Cuantificar el odio, contando el número de insultos (fuck, idiot, stupid, retard, moron, nutjob...) y calcular porcentajes para saber a qué partido iban más dirigidos

Obtener un datafreme con el porcentaje de insultos a cada partido por cada país.

1. La idea es combianar los ejercicios anteriores, seleccionar los tweets por país filtrando los nulos.
2. Añadir una columna democrat: si en el Tweet aparece obama o democrat -> Democrat, sino -. El texto está en minúsculas.
3. Añadir una columna republican: si en el Tweet aparece romney, republican o tcot -> Republican, sino -. El texto está en minúsculas.
4. Añadir una columna insult: si en el Tweet aparece fuck, idiot, stupid, retard, moron o nutjob -> Yes, sino No. El texto está en minúsculas.
5. Nos quedamos solo con los tweets en los que haya insultos, insult == YES.
6. Concatenar las columnas democrat y republican, obteniendo las combinaciones. Agrupar por país y esta última columna creada, obteniendo el porcentaje de insultos.

In [ ]:
#Nos quedamos con los datos que queremos, filtramos paises nulos 
df2 = df.select(F.col('text'), F.col('country')) \
                 .filter(F.col('country').isNotNull()) 

#Añadidos columna democrat.
columnaDemocratDF = df2.withColumn('democrat', F.when(F.lower(F.col('text')).like('%obama%'), "Democrat") \
                        .otherwise(F.when(F.lower(F.col('text')).like('%democrat%'), "Democrat") \
                              .otherwise("-")))

#Añadidos columna republican.
columnaRepublicanDemocratDF = columnaDemocratDF.withColumn('republican', F.when(F.lower(F.col('text')).like('%romney%'), "Republican") \
                                      .otherwise(F.when(F.lower(F.col('text')).like('%republican%'), "Republican") \
                                          .otherwise(F.when(F.lower(F.col('text')).like('%tcot%'), "Republican") \
                                              .otherwise("-"))))

#Añadidos columna insult.
columnainsultDF = columnaRepublicanDemocratDF.withColumn('insult', F.when(F.lower(F.col('text')).like('%fuck%'), "YES") \
                                                 .otherwise(F.when(F.lower(F.col('text')).like('%idiot%'), "YES") \
                                                      .otherwise(F.when(F.lower(F.col('text')).like('%stupid%'), "YES") \
                                                          .otherwise(F.when(F.lower(F.col('text')).like('%retard%'), "YES") \
                                                              .otherwise(F.when(F.lower(F.col('text')).like('%moron%'), "YES") \
                                                                  .otherwise(F.when(F.lower(F.col('text')).like('%nutjob%'), "YES") \
                                                                      .otherwise("NO")))))))

#Seleccionamos las columnas que nos interesan e insult == YES.
insultYesDF = columnainsultDF.select('country', 'republican', 'democrat', 'insult') \
                             .filter(F.col('insult') == 'YES')

#Concatenamos las columnas de partido, agrupamos país y la concatenada ordenada por país el porcentaje de insultos.
dfLast = insultYesDF.withColumn('concatenated', F.concat(*['republican', 'democrat'])) \
                    .groupBy('country', 'concatenated') \
                    .count() \
                    .select('country', 'concatenated', (100*F.col('count')/nTweets).alias('count')) \
                    .sort('country', ascending=False)

#Mostramos el último dataframe.
dfLast.show()

Escribir las conclusiones

# Análisis final

### Ejercicio 1

Calcula el número de tweets por día y estudiar que día de la semana está la gente más activa en twiter. Quedarse solo con el día con más interacciones

In [ ]:
weetsByDay = ###RELLENAR
weetsByDay

### Ejercicio 2

Estudia el uso de twiter segun la hora del día. Mostrar solo los 10 primeros. 

In [ ]:
histTweetsHour = ###RELLENAR
histTweetsHour

No se aprecia mucha información en la tabla mostrada, sin embargo sí nos ayuda a entender mejor el ejercicio anterior ya que vemos que efectivamente el día 10 de septiembre posee información de todas las horas (menos 15 y 16h) del día mientras que el resto de días no y esto lleva a que este día pareciera en el ejercicio anterior que había tenido más actividad.

Para poder estudiar mejor el uso de twiter hora a hora vamos a filtrar por el día 10 y vamos a dibujar un histograma.


In [ ]:
histTweetsHour = ###RELLENAR

In [ ]:
(x_values, y_values) = zip(*histTweetsHour)
plt.bar(x_values, y_values)
plt.title('Histograma de tweets/hora')
plt.xlabel('N. tweets')
plt.ylabel('N. hora del día')
plt.show()

Se observan dos picos y dos valles. Los picos coinciden con la noche y con el medio día (la hora de comer) aunque el segundo pico es mucho menor que el primero. Los valles se corresponden con las horas centrales de trabajo por la mañana o por la tarde (aunque no tenemos información entre las 14 y las 17h) y con las últimas horas de la madrugada. Se observa que a partir de las 2 a.m. los usuarios se van iendo a dormir y el número de tweets disminuye progresivamente hasta la hora de comer.


### Ejercicio 3

Estudia el número de tweets relacionados con cada partido, que tengan la palabra (o partícula) "win" y la palabra o partícula "lose".

En primer lugar concluimos que no hay muchos tweets que contengan las palabras "win" y "lose". Al margen de esto podemos ver que por lo general hay una mayor tendencia hacia considerar a un partido ganador que al otro perdedor, es decir, predominan los tweets de refuerzo positivo. Por otro lado destacar que hay mas tweets que consideran ganador al partido demócrata pero también es mayor el número de los que consideran que van a perder, por lo que es dificil extraer conclusiones.


In [ ]:
columnaDemocratDF = ###RELLENAR

In [ ]:
columnaRepublicanDemocratDF = ###RELLENAR

In [ ]:
columnastatusDF = ###RELLENAR

In [ ]:
dfLast = ###RELLENAR
dfLast

In [ ]:
round_fun = lambda res: list(map(lambda k_v: (k_v[0],k_v[1], np.round(k_v[2], 4)), res))
round_fun(dfLast.collect())

### Ejercicio 4

Estudia los 10 usuarios tienen un mayor número de seguidores en sus tweets.

Se observa que el usuario con más seguidores fue Barack Obama (condidato demócrata) y que el candidatos republicano ni siquiera aparece en el top10. Esto demuestra que Obama hizo un mayor uso de las redes sociales (en concreto tweeter) para llegar a sus votantes y esta pudo ser la clave de su éxito. En el resto del top abundan los canales de noticias y alguna otra personalidad individual.

Como observación decir que la columna de followers_count es muy variable, lo cual es extraño. Por ejemplo Barack Obama puede pasar de 600 a 2000 seguidores en menos de 10 minutos.


In [ ]:
df.printSchema()

In [ ]:
dfusersFollowers = ###RELLENAR
dfusersFollowers

### Ejercicio 5

Estudia la correlación entre el número de seguidores y los retweets. (Es extraño que la correlación sea negativa porque implica que a mayor número de followers menor numero de tweets. Es posible que esto sea debido a que la columna followers_count sea muy variable y tenga alguna incoherencia)

In [ ]:
correlation = ###RELLENAR
correlation

### Ejercicio 6

Estudiar cuales son los países con mayor número de followers. Para ello agrupar por países, habiendo filtrado los países que son Nulos. Mostrar por pantalla los 10 primeros y los 10 ultimos.

In [ ]:
paisesDesc = ###RELLENAR

In [ ]:
paisesAsc = ###RELLENAR

Los paises con mayor población cuentan con mayor número de tweets sin embargo India es una excepción. Esto se debe también al idioma utilizado.

### Ejercicio 7

El siguiente estudio propone determinar la relacion entre los usuarios con más folowers (influencers) y las menciones de bandos políticos como pueden ser los demócratas de Obama y los republicanos de Rommey. Para ello se contará el número de tweets mencionando a alguno de los dos partidos entre Top 10 influencers de USA.

In [ ]:
#Dataframe auxiliar que cuenta con la información de los usuarios requerida
aux = ###RELLENAR

#Añadidos columna democrat.
columnaDemocratDF = ###RELLENAR

#Añadidos columna republican.
columnaRepublicanDF = ###RELLENAR

#Unimos los df, agrupamos las columnas democrat - repúblican y ordenamos la agrupación respecto al número de seguidores de los usuarios descendentemente
dfLast = ###RELLENAR

#Nos quedamos con el top10 tweets (cada fila es un tweet)
dfLast.###RELLENAR
dfLast

Se observa que los usuarios con mayor número de seguidores, no twittean sobre política, y si lo hacen suele ser mencionando a ambos partidos. Los dos últimos tweets hacen referencia a los demócratas, sin embargo no sabemos si es para bien o para mal (para ello deberíamos pasar el filtro de insultos previo).

Este estudio no es extrapolable puesto que contamos con una población pequeña de usuarios y tan solo estamos teniendo en cuenta 10 tweets.
